In [1]:
import pickle
from pathlib import Path
import numpy as np
import gurobipy as gp; gp.Model()

from src.input import Input_uc, Input_suc, get_min_up_down_initials
from src.output import Output_uc
from src.unit_commitment import solve_uc
from src.stochastic_unit_commitment import solve_suc

Set parameter Username
Set parameter LicenseID to value 2659802
Academic license - for non-commercial use only - expires 2026-05-01


In [2]:
def load_data():
    path_folder_processed = Path.cwd() / "data" / "input" / "processed"
    timestamp_2022 = np.load(path_folder_processed / "timestamp_2022.npy")
    
    global num_units, num_periods, time_start
    num_units = 122
    num_periods = 24
    time_start, time_end = np.datetime64("2022-07-21T00"), np.datetime64("2022-07-21T23")
    idx_time_start, idx_time_end = np.where(timestamp_2022 == time_start)[0][0], np.where(timestamp_2022 == time_end)[0][0]

    global unit_type, idx_nuclear, idx_coal, idx_lng
    unit_type = np.load(path_folder_processed / "unit_type.npy")
    idx_nuclear = np.load(path_folder_processed / "idx_nuclear.npy")
    idx_coal = np.load(path_folder_processed / "idx_coal.npy")
    idx_lng = np.load(path_folder_processed / "idx_lng.npy")

    global p_min, p_max, ramp_up, ramp_down, startup_ramp, shutdown_ramp, min_up, min_down
    p_min = np.load(path_folder_processed / "p_min.npy")
    p_max = np.load(path_folder_processed / "p_max.npy")
    ramp_up = np.load(path_folder_processed / "ramp_up.npy")
    ramp_down = np.load(path_folder_processed / "ramp_down.npy")
    startup_ramp = np.load(path_folder_processed / "startup_ramp.npy")
    shutdown_ramp = np.load(path_folder_processed / "shutdown_ramp.npy")
    min_up = np.load(path_folder_processed / "min_up.npy")
    min_down = np.load(path_folder_processed / "min_down.npy")

    global cost_lin, cost_const, cost_startup_step_old, num_cooling_steps_old, cost_startup_step, step_length
    cost_lin = np.load(path_folder_processed / "cost_lin.npy")
    cost_const = np.load(path_folder_processed / "cost_const.npy")
    cost_startup_step_old = pickle.load(open(path_folder_processed / "cost_startup_step_old.pkl", "rb"))
    num_cooling_steps_old = np.load(path_folder_processed / "num_cooling_steps_old.npy")
    cost_startup_step = np.load(path_folder_processed / "cost_startup_step.npy")
    step_length = np.load(path_folder_processed / "step_length.npy")

    global demand, renewable
    demand = np.load(path_folder_processed / "demand_2022.npy")[idx_time_start:idx_time_end+1]
    renewable = np.load(path_folder_processed / "renewable_gen_2022.npy")[idx_time_start:idx_time_end+1]

    global p_prev, u_prev, min_up_r, min_down_r, min_up_0, min_down_0
    p_prev = np.load(path_folder_processed / "p_prev.npy")
    u_prev = np.load(path_folder_processed / "u_prev.npy")
    min_up_r = np.load(path_folder_processed / "min_up_r.npy")
    min_down_r = np.load(path_folder_processed / "min_down_r.npy")
    min_up_0 = np.load(path_folder_processed / "min_up_0.npy")
    min_down_0 = np.load(path_folder_processed / "min_down_0.npy")

    global thermal_demand_scenario
    thermal_demand_scenario = np.load(path_folder_processed / "thermal_demand_2022721_tk.npy")

In [3]:
load_data()

In [4]:
input_uc = Input_uc(
    # meta
    unit_type=unit_type,
    idx_nuclear=idx_nuclear,
    idx_coal=idx_coal,
    idx_lng=idx_lng,
    num_units=num_units,
    num_periods=num_periods,
    # system
    demand=np.zeros(num_periods), # not used # input_uc is just cus i don't have time to add these liens to input_suc
    reserve=np.zeros(num_periods),
    renewable=np.zeros(num_periods),
    # generator
    p_min=p_min,
    p_max=p_max,
    ramp_up=ramp_up,
    ramp_down=ramp_down,
    startup_ramp=startup_ramp,
    shutdown_ramp=shutdown_ramp,
    min_up=min_up,
    min_down=min_down,
    # cost function - generation
    cost_lin=cost_lin,
    cost_const=cost_const,
    # cost function - startup (paper formulation)
    cost_startup_step=cost_startup_step,
    step_length=step_length,
    # cost function - startup (lecture note formulation)
    cost_startup_step_old=cost_startup_step_old,
    num_cooling_steps_old=num_cooling_steps_old,
    # initial conditions
    p_prev=p_prev,
    u_prev=u_prev,
    min_up_r=min_up_r,
    min_down_r=min_down_r,
    min_up_0=min_up_0,
    min_down_0=min_down_0,
)

input_suc = Input_suc(
    demand_fore=demand,# testing that forecast = actual
    renewable_fore=renewable, # testing that forecast = actual
    num_scenarios=6,
    thermal_demand_scenario=thermal_demand_scenario,
    scenario_p_weight=np.ones(6) / 6,
    voll=3500000,
    fr_margin_pu=0.02,
)


In [ ]:
output_suc = Output_uc()
solve_suc(
    input_uc=input_uc,
    input_suc=input_suc,
    output_suc=output_suc,
    verbose=True,
)

Set parameter OutputFlag to value 1
Set parameter Symmetry to value 2
Set parameter PreDual to value 2
Set parameter Presolve to value 1
Set parameter PreSparsify to value 2
Set parameter Disconnected to value 2
Set parameter Heuristics to value 1
Set parameter ProjImpliedCuts to value 2
Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (linux64 - "Arch Linux")

CPU model: 12th Gen Intel(R) Core(TM) i7-12700, instruction set [SSE2|AVX|AVX2]
Thread count: 20 physical cores, 20 logical processors, using up to 20 threads

Non-default parameters:
Disconnected  2
Heuristics  1
Symmetry  2
ProjImpliedCuts  2
PreDual  2
Presolve  1
PreSparsify  2

Optimize a model with 105622 rows, 38376 columns and 379337 nonzeros
Model fingerprint: 0xf875783f
Variable types: 20808 continuous, 17568 integer (17568 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+03]
  Objective range  [6e+02, 6e+07]
  Bounds range     [1e+00, 1e+03]
  RHS range        [1e+00, 9e+04]
Sparsify removed 2170 nonzeros